In [1]:
import pandas as pd
#pd.set_option("display.max_rows", None)
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import dash
import dash_core_components as dcc
import dash_html_components as html

import ibmdata

# Plot HeatMap

In [2]:
import copy
def create_z(df,x_col,y_col,z_col):
    # create an empty annotations container
    if len(df['family_code'].unique()) == 1:
        if df['family_code'].unique() == 'Q5':
            annotations = [['', '', '', 0, 0, 0, 0, 0, '', '', ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', '', '', 0, 0, 0, 0, 0, '', '', '']]
            # Fill the annotations container
            for i in range(df.shape[0]):
                x = int(df.iloc[i][x_col] - 1)
                y = int(df.iloc[i][y_col] - 2)
                annotations[y][x] = df.iloc[i][z_col].round(2)
        elif df['family_code'].unique() == 'Q6':
            annotations = [['', '', '', '', 0, 0, 0, '', '', '', ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', '', '', '', 0, 0, 0, '', '', '', '']]
            # Fill the annotations container
            for i in range(df.shape[0]):
                x = int(df.iloc[i][x_col] - 2)
                y = int(df.iloc[i][y_col] - 2)
                annotations[y][x] = df.iloc[i][z_col].round(2)
        elif df['family_code'].unique() == 'XQ':
            annotations = [['', '', '', 0, 0, 0, 0, 0, '', '', ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', '', '', 0, 0, 0, 0, 0, '', '', '']]
            # Fill the annotations container
            for i in range(df.shape[0]):
                x = int(df.iloc[i][x_col] - 1)
                y = int(df.iloc[i][y_col] - 2)
                annotations[y][x] = df.iloc[i][z_col].round(2)
    else:
        return False, False

    # Replace '' with a value to plot with
    z = copy.deepcopy(annotations)
    for y, row in enumerate(z):
        for x, value in enumerate(row):
            if z[y][x] == '':
                if min(df[z_col]) <= 0:
                    z[y][x] = min(df[z_col])-1
                else:
                    z[y][x] = -1
        
    return z, annotations


def plot_heatmap(df,x_col,y_col,z_col):
    z, annotation = create_z(df,x_col=x_col,y_col=y_col,z_col=z_col)
    if z == False:
        return
    else:
        if df['family_code'].unique() == 'Q5':
            y = [i for i in range(2,11)]
        elif df['family_code'].unique() == 'Q6':
            y = [i for i in range(2,12)]
        elif df['family_code'].unique() == "XQ":
            y = [i for i in range(2,12)]
        fig = ff.create_annotated_heatmap(
            z=z,
            y = y,
            x = [i for i in range(1,12)],
            annotation_text=annotation,
            colorscale = [(0.00, "white"),(0.01,'rgb(255,247,236)'),(1.00, 'rgb(127, 0, 0)')]
        )
        fig.update_layout(
            xaxis_title="retx",
            yaxis_title="rety",
            width = 680,
            height = 600,
            autosize = False
        )
        fig.update_xaxes(side="bottom")
        return fig

# Pull data

In [3]:
def fill_zeros(df):
    data = {'lot_id':[],'wafer_id':[],'family_code':[],'sail_date':[],'retx':[],'rety':[],}
    temp_df = df.groupby(['lot_id','wafer_id','family_code'])['sail_date'].min().reset_index()
    for row in temp_df.drop_duplicates().iterrows():
        if row[1][2] == 'Q5':
            data['lot_id'].extend([row[1][0] for i in range(75)])
            data['wafer_id'].extend([row[1][1] for i in range(75)])
            data['family_code'].extend([row[1][2] for i in range(75)])
            data['sail_date'].extend([row[1][3] for i in range(75)])
            data['retx'].extend([1,1,1,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,
                        6,6,6,6,6,6,6,6,6,7,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,9,9,9,9,9,9,
                        9,10,10,10,10,10,11,11,11])
            data['rety'].extend([5,6,7,4,5,6,7,8,3,4,5,6,7,8,9,2,3,4,5,6,7,8,9,10,2,3,4,5,6,7,8,9,
                        10,2,3,4,5,6,7,8,9,10,2,3,4,5,6,7,8,9,10,2,3,4,5,6,7,8,9,10,3,4,5,
                        6,7,8,9,4,5,6,7,8,5,6,7])
        elif row[1][2] == 'Q6':
            data['lot_id'].extend([row[1][0] for i in range(80)])
            data['wafer_id'].extend([row[1][1] for i in range(80)])
            data['family_code'].extend([row[1][2] for i in range(80)])
            data['sail_date'].extend([row[1][3] for i in range(80)])
            data['retx'].extend([2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,6,6,6,6,6,6,6,
                        6,6,6,7,7,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,10,
                        10,10,10,10,10,10,10,11,11,11,11,11,11,12,12])
            data['rety'].extend([5,6,7,8,4,5,6,7,8,9,3,4,5,6,7,8,9,10,3,4,5,6,7,8,9,10,2,3,4,5,6,7,
                        8,9,10,11,2,3,4,5,6,7,8,9,10,11,2,3,4,5,6,7,8,9,10,11,3,4,5,6,7,8,9,10,3,4,5,
                        6,7,8,9,10,4,5,6,7,8,9,6,7])
        elif row[1][2] == 'XQ':
            data['lot_id'].extend([row[1][0] for i in range(86)])
            data['wafer_id'].extend([row[1][1] for i in range(86)])
            data['family_code'].extend([row[1][2] for i in range(86)])
            data['sail_date'].extend([row[1][3] for i in range(86)])
            data['retx'].extend([1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,
                        6,6,6,6,6,6,6,6,6,6,7,7,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,
                        10,10,10,10,10,10,11,11,11,11])
            data['rety'].extend([5,6,7,8,4,5,6,7,8,9,3,4,5,6,7,8,9,10,2,3,4,5,6,7,8,9,10,11,2,3,4,5,6,7,
                        8,9,10,11,2,3,4,5,6,7,8,9,10,11,2,3,4,5,6,7,8,9,10,11,2,3,4,5,6,7,8,9,10,11,3,4,5,
                        6,7,8,9,10,4,5,6,7,8,9,5,6,7,8])
    filler_df = pd.DataFrame(data)
    output_df = filler_df.merge(df,how='left',on=['lot_id','wafer_id','family_code','retx','rety']).fillna(0)
    output_df['sail_date'] = [output_df.iloc[i]['sail_date_x'] if output_df.iloc[i]['sail_date_y'] == 0 else output_df.iloc[i]['sail_date_y'] for i in range(len(output_df))]
    output_df = output_df.drop(columns=['sail_date_x','sail_date_y'])
    return output_df

In [4]:
DAYSBACK = 365
GF_QUERY = f"""
WITH cte AS(
    (SELECT lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X - 1 AS retx, unitcell_Y AS rety,
        CASE WHEN LENGTH(parm_Label) = 37 THEN tw.testProgramName||'_0p75'||RIGHT(parm_Label,11) ELSE tw.testProgramName||'_0p75'||RIGHT(parm_Label,12) END AS label,
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL11','SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,10,17)) = 'SSCANLATCH0P75VFM')
        AND ((unitcell_X - 1) || '_' || unitcell_Y NOT IN ('4_12','5_12','6_12','7_12','3_11','9_11','2_10','10_10','1_9','11_9','1_4','11_4','2_3','10_3','3_2','9_2','4_1','5_1','7_1'))
        AND parmYield < 100
        AND family_Code = 'XQ'
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        --AND wafer_id = 'AZ6B5-19'
    ORDER BY lot_Id, wafer_Id, parm_Label)
)
SELECT lot_id, wafer_id, family_Code, MIN(date) AS sail_date, retx, rety,
    SUM(
        CASE 
            WHEN Sail = 'SSL11' AND RIGHT(label,10) IN ('block6_so0','block6_so5','block7_so0','block7_so5') THEN 1
            WHEN Sail = 'SSL11' AND SUBSTR(label,11,7) IN ('block22','block24','block25') THEN 1
            WHEN Sail = 'SSL11' AND (RIGHT(label,10) NOT IN ('block6_so0','block6_so5','block7_so0','block7_so5') OR SUBSTR(label,11,7) NOT IN ('block22','block24','block25')) THEN -0.3
            ELSE 0 END
        ) AS SSL1_GF_metric,
    SUM(
        CASE 
            WHEN Sail = 'SSL21' AND RIGHT(label,10) IN ('block6_so0','block6_so5','block7_so0','block7_so5') THEN 1
            WHEN Sail = 'SSL21' AND SUBSTR(label,11,7) IN ('block22','block24','block25') THEN 1
            WHEN Sail = 'SSL21' AND (RIGHT(label,10) NOT IN ('block6_so0','block6_so5','block7_so0','block7_so5') OR SUBSTR(label,11,7) NOT IN ('block22','block24','block25')) THEN -0.3
            ELSE 0 END
        ) AS SSL2_GF_metric,
    SUM(
        CASE 
            WHEN Sail = 'SSL31' AND RIGHT(label,10) IN ('block6_so0','block6_so5','block7_so0','block7_so5') THEN 1
            WHEN Sail = 'SSL31' AND SUBSTR(label,11,7) IN ('block22','block24','block25') THEN 1
            WHEN Sail = 'SSL31' AND (RIGHT(label,10) NOT IN ('block6_so0','block6_so5','block7_so0','block7_so5') OR SUBSTR(label,11,7) NOT IN ('block22','block24','block25')) THEN -0.3
            ELSE 0 END
        ) AS SSL3_GF_metric
FROM cte
GROUP BY lot_id, wafer_id, family_code, retx, rety
ORDER BY MIN(date) DESC, lot_id, cte.wafer_id
"""

In [5]:
def ghost_fin_chip():
    df = ibmdata.isdw.query(GF_QUERY)
    df['ssl1_gf_metric'] = df['ssl1_gf_metric'].astype(float).round(2)
    df['ssl2_gf_metric'] = df['ssl2_gf_metric'].astype(float).round(2)
    df['ssl3_gf_metric'] = df['ssl3_gf_metric'].astype(float).round(2)
    df['GF_metric'] = df['ssl1_gf_metric']+df['ssl2_gf_metric']+df['ssl3_gf_metric']
    df['GF_metric'] = df['GF_metric'].round(2)
    df['Ghost_Fin'] = df['GF_metric'].apply(lambda x: 1 if x > 1 else 0)
    df = fill_zeros(df)
    return df

In [6]:
def ghost_fin_wafer():
    df = ghost_fin_chip()
    agg_df = df.groupby(['lot_id','wafer_id','family_code']).agg({'sail_date':'min', 'Ghost_Fin':'sum'}).reset_index()
    Q5_df = agg_df[agg_df['family_code']=='Q5']
    Q5_df['Ghost_Fin_YieldLoss'] = Q5_df['Ghost_Fin'].apply(lambda x: 100*x/75).round(2)
    Q6_df = agg_df[agg_df['family_code']=='Q6']
    Q6_df['Ghost_Fin_YieldLoss'] = Q6_df['Ghost_Fin'].apply(lambda x: 100*x/80).round(2)
    XQ_df = agg_df[agg_df['family_code']=='XQ']
    XQ_df['Ghost_Fin_YieldLoss'] = XQ_df['Ghost_Fin'].apply(lambda x: 100*x/86).round(2)
    agg_df = pd.concat([Q6_df,Q5_df,XQ_df])
    agg_df = agg_df.rename(columns={"Ghost_Fin": "Ghost_Fin_ChipCount"})
    agg_df = agg_df.sort_values('sail_date',ascending=False).reset_index(drop=True)
    return agg_df

In [7]:
def ghost_fin_lot():
    df = ghost_fin_wafer()
    agg_df = df.groupby(['lot_id','family_code']).agg({'sail_date':'min','Ghost_Fin_ChipCount':'mean', 'Ghost_Fin_YieldLoss':'mean'}).reset_index()
    agg_df = agg_df.rename(columns={"Ghost_Fin_ChipCount": "Avg_Ghost_Fin_ChipCount","Ghost_Fin_YieldLoss":"Avg_Ghost_Fin_YieldLoss" })
    agg_df['Avg_Ghost_Fin_ChipCount'] = agg_df['Avg_Ghost_Fin_ChipCount'].astype(float).round(2)
    agg_df['Avg_Ghost_Fin_YieldLoss'] = agg_df['Avg_Ghost_Fin_YieldLoss'].astype(float).round(2)
    agg_df = agg_df.sort_values('sail_date',ascending=False).reset_index(drop=True)
    return agg_df

In [12]:
df = ghost_fin_chip()

In [13]:
df.to_csv("~/Downloads/tmp.csv")

In [46]:
df[df['lot_id'] == 'AZBD5.1']

,lot_id,wafer_id,family_code,retx,rety,ssl1_gf_metric,ssl2_gf_metric,ssl3_gf_metric,GF_metric,Ghost_Fin,sail_date
41108,AZBD5.1,AZBD5-08,XQ,1,5,0.0,-0.3,0.0,-0.3,0.0,2021-12-08
41109,AZBD5.1,AZBD5-08,XQ,1,6,0.0,-0.3,0.0,-0.3,0.0,2021-12-08
41110,AZBD5.1,AZBD5-08,XQ,1,7,0.0,0.0,0.0,0.0,0.0,2021-12-08
41111,AZBD5.1,AZBD5-08,XQ,1,8,0.0,0.0,0.0,0.0,0.0,2021-12-08
41112,AZBD5.1,AZBD5-08,XQ,2,4,0.0,0.0,0.0,0.0,0.0,2021-12-08
...,...,...,...,...,...,...,...,...,...,...,...
41361,AZBD5.1,AZBD5-12,XQ,10,9,0.0,0.0,0.0,0.0,0.0,2021-12-08
41362,AZBD5.1,AZBD5-12,XQ,11,5,0.0,0.0,0.0,0.0,0.0,2021-12-08
41363,AZBD5.1,AZBD5-12,XQ,11,6,0.0,0.0,0.0,0.0,0.0,2021-12-08
41364,AZBD5.1,AZBD5-12,XQ,11,7,0.0,0.0,0.0,0.0,0.0,2021-12-08


# Plotting

In [49]:
#lots = ['AZBD5.1']
#wafer_df = df[df.lot_id.isin(lots)]
wafers = ['AZBD5-11']
wafer_df = df[df.wafer_id.isin(wafers)]
wafer_df = wafer_df.groupby(['family_code','retx','rety']).mean().reset_index()
fig = plot_heatmap(wafer_df,'retx','rety','GF_metric')

In [50]:
fig

In [10]:
tmp= ghost_fin_wafer()

In [11]:
tmp.to_csv("~/Downloads/tmp.csv")

In [49]:
tmp

,lot_id,wafer_id,family_code,sail_date,Ghost_Fin_ChipCount,Ghost_Fin_YieldLoss
0,AZBBDA.1,AZBBD-21,XQ,2021-10-09,0.0,0.00
1,AZBBDA.1,AZBBD-22,XQ,2021-10-09,1.0,1.16
2,AZBCG.1,AZBCG-25,XQ,2021-10-07,0.0,0.00
3,AZBCG.1,AZBCG-05,XQ,2021-10-07,0.0,0.00
4,AZBCG.1,AZBCG-16,XQ,2021-10-07,0.0,0.00
5,AZBCG.1,AZBCG-14,XQ,2021-10-07,0.0,0.00
6,AZBCG.1,AZBCG-13,XQ,2021-10-07,0.0,0.00
7,AZBCG.1,AZBCG-12,XQ,2021-10-07,0.0,0.00
8,AZBCG.1,AZBCG-11,XQ,2021-10-07,0.0,0.00
9,AZBCG.1,AZBCG-10,XQ,2021-10-07,1.0,1.16
